In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math
from IPython.core.display import display,HTML
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
    
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
import plotly
plotly.__version__

In [7]:
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'Dropbox_datafile_complete_new_ranking.csv'
#input_file = 'Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_population_density_when_available_career_stage_GINI_folder_act_categories.csv'
df = pd.read_csv(path+input_file, sep=';',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
df.drop('Unnamed: 0', axis=1, inplace=True)

print df.shape







######## load dictionary aggregated info by user  
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_user_attr_more_var.pickle'
with open(pickle_name, 'rb') as handle:
    dict_user_id_user_attr = pickle.load(handle)
print len(dict_user_id_user_attr.keys())

df_users = pd.DataFrame.from_dict(dict_user_id_user_attr,orient='index')
#df_selection_users=df_folders[df_folders['user_univ_ranking']<=50]
print df_users.shape   # 440353, 11







(1403349, 38)
440353
(440353, 14)


In [22]:
df_users.columns
print df_users.user_tot_act.mean(), df_users.user_tot_act.quantile(q=0.6)
act_threshold=60  #  28 is the 60% ,  56 is the 65%  ,   168 is the 70% percentile (50% percentile is 0)

27029.3620573 28.0


In [ ]:
# for item in df.email_domain.unique():
#     print item,"   "

In [14]:
# #### time window for temporal networks
# string_start_date='2014-05-01'
# start_date=pd.Timestamp(string_start_date)

# #string_end_date='2014-11-01'
# end_date=start_date + pd.Timedelta('180 days')




# list_tupla_dates=[]
# for i in range(6):
#     print start_date, end_date    
#     tupla_dates=[start_date,end_date]
#     list_tupla_dates.append(tupla_dates)

    
#     start_date += pd.Timedelta('180 days')
#     end_date += pd.Timedelta('180 days')

# # 2014-05-01_2014-10-28

In [ ]:

### for plotting time series directly from data
#df.plot(x='folder_creation_date',y='folder_lifespan',ls='',marker='.') 



In [ ]:

G=nx.Graph()
dict_folder_list_users={}
dict_folder_dict_user_act_in_folder={}

    
for row in tqdm_notebook(df.iterrows()):

        folder=row[1]["folder_id"]
        user_id=row[1]["user_id"]                           
        
        try: 
            dict_folder_list_users[folder].append(user_id)
        except:
            dict_folder_list_users[folder]=[]
            dict_folder_list_users[folder].append(user_id)
     
print "done with the dict., size:",len(dict_folder_list_users)



for folder in dict_folder_list_users:
        dict_folder_list_users[folder]=list(set(dict_folder_list_users[folder]))  # remove possible duplicates      

        lista=dict_folder_list_users[folder]

        if len(lista)>1:

            lista_pares=itertools.combinations(lista, 2)        

            for item in lista_pares:
                e1=item[0]
                e2=item[1]
                G.add_edge(e1,e2)
                
                try: 
                    G.edge[e1][e2]["num_common_projects"] +=1
                except KeyError:
                    G.edge[e1][e2]["num_common_projects"] =1
                    
#                 print e1, e2, G.edge[e1][e2]["num_common_folders"]    
#                 raw_input()
               # print "added edge:",e1, e2
    
        else:
            n=lista[0]
            G.add_node(n)
            G.node[n]["tot_act"]
            

print "  N:", len(G.nodes()),"  L:", len(G.edges())       
GC = max(nx.connected_component_subgraphs(G), key=len)
print "\n  GC:     N:", len(GC.nodes()), "  L:", len(GC.edges())






#### the master dict is larger than G.nodes(), and the set_node_attribute does not work in that case, so i get a selection of the dict only with the common nodes:
#######################

# new_dict_user_id = {k: dict_user_id_domain[k] for k in set(G.nodes()) & set(dict_user_id_domain.keys())}
# print len(G.nodes()), len(new_dict_user_id)
# # i add the email as node attributes    
# nx.set_node_attributes(G, 'email', new_dict_user_id)
# ############





path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/"
filename="first_network_all_new.pickle"
with open(path+filename,'wb') as f:
    pickle.dump(G, f)
print "written:",path+filename




